# l'Objectif du projet:
* Prédire la qualité du vin vis-à-vis des consommateurs 
* Aider les consommateur à prendre une decision par rapport à la qualité du vin pour son utilisation
* aider les entreprises fabricatrices à rendre la qualité du vin mailleur, en se basant sur les composant constituant le vin

# De cette façon, nous allons:
* importer les données
* visualiser les données
* faire une analyse exploratoire de ses données
* tester les donnéesdonnées en les entraînant à des modèles d'entraînement de machin Learning
* rendre lesdonnées explicite pour que les deux parties parviennent à prendre les décision qui les conviennent.

## Nous classifions nos données comme suit:
* utilisons les colonne comme de variables à predir
* choisisson parmis ces variables les mieux
* utilison ces variables pour la prediction

# Voici nos variacbles:

 *  volatile acidity    
 *  citric acid         
 *  residual sugar      
 *  chlorides           
 *  free sulfur dioxide 
 *  total sulfur dioxide
 *  density             
 *  pH                  
 *  sulphates           
 *  alcohol             
 *  quality
 
 On se base sur:  (residual sugar,  pH, quality) pour prédire la qualité de vin


## Concretisation du projet

* On importe toutes les bibliothèques naicessaires

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from seaborn.axisgrid import plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# On importe notre fichier de données pour faire les analyses possibles

In [58]:
# Lire les données
def read_data(file_path):
    data = pd.read_csv(file_path)
    return data
data = read_data('winequality-red.csv')

* Connaissance de lastructure de notre données

In [ ]:
data.info()

Affichage de données 

In [ ]:
data.head()

# Cette ligne de code calcule les fréquences relatives des différentes valeurs de la colonne quality dans le dataframe data. 

In [ ]:
data["quality"].value_counts(normalize=True)

Les résultats indiquent que la valeur la plus fréquente dans la colonne quality est 5, qui représente environ 42.6% des valeurs. La deuxième valeur la plus fréquente est 6, qui représente environ 39.9% des valeurs. Les autres valeurs (7, 4, 8 et 3) sont moins fréquentes et représentent respectivement environ 12.4%, 3.3%, 1.1% et 0.6% des valeurs.

* L'histogramme de chaque composant du vin

In [ ]:
data.hist(figsize=(20,10))

# Consideron la meilleur qualité du vin est: qualité >= 7 

In [ ]:
data[data.quality >= 7].describe()

# Mettons en relation les données pour mieux predire la meilleur qualité du vin

In [ ]:
data.corr()

* Le code suivant nous spécifie la corrélation

In [ ]:

sns.catplot(data.corr())

# le code suivant:
* nous affiche les données corrélées sous forme de cube corrélés

In [ ]:

plt.figure(figsize=(12,10))
mask = np.triu(np.ones_like(data.corr(), dtype=bool))
sns.heatmap(data.corr(), mask=mask,center=0,cmap='RdBu', linewidths=1, annot=True)
plt.title('Données corrélées ou Coefficient de corrélation')

# Cherchon les données manquantes pour les corrigées

In [ ]:
data.isna().sum()

* *  cette fonction trouve les valeurs aberrantes dans un DataFrame pandas en utilisant la méthode des écarts interquartiles (IQR) et renvoie un nouveau DataFrame contenant toutes les lignes avec des valeurs aberrantes.

In [ ]:
def find_outliers(data):
    outliers = pd.DataFrame(columns=data.columns)
    for col in data.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        col_outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
        outliers = pd.concat([outliers, col_outliers], axis=0)
    return outliers.drop_duplicates()

outliers = find_outliers(data)
print(outliers)

Affichons le nombre des lignes et des colonnes des valeurs aberrantes

In [122]:
num_outlier_rows = outliers.shape[0]
num_outlier_cols = outliers.count().sum()

print(f'Nombre de lignes contenant des valeurs aberrantes :{num_outlier_rows}')
print(f'Nombre de colonne contenant des valeurs aberrantes : {num_outlier_cols}')

Nombre de lignes contenant des valeurs aberrantes :434
Nombre de colonne contenant des valeurs aberrantes : 5208


# Traitons les valeurs aberrante s'ils existent
Ce code calcule d’abord les premier et troisième quartiles des données, 
puis utilise ces valeurs pour calculer l’écart interquartile (IQR).
Ensuite, il définit une limite inférieure et supérieure pour détecter les valeurs aberrantes
en utilisant la formule lower_bound = Q1 - 1.5 * IQR et upper_bound = Q3 + 1.5 * IQR.
Enfin, il remplace les valeurs aberrantes par la médiane des données.

In [ ]:
def remove_outliers(data):
    # Calculer le premier et le troisième quartile
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    
    # Calculer l'écart interquartile (IQR)
    IQR = Q3 - Q1
    
    # Définir la limite inférieure et supérieure pour détecter les valeurs aberrantes
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remplacer les valeurs aberrantes par la médiane
    data[data < lower_bound] = np.median(data)
    data[data > upper_bound] = np.median(data)
    
    return data

data = remove_outliers(data)
data

Voici les etapes utilisés pour analyser nos données dand le fichier data:

* Charger les données: 
On a utilisé la binliothèque pandas pour charger les données à partir du fichier data dans un dataframe.

* Examiner la structure des données: 
On utilise en suite head(), tail, describe et info, pour examiner la structure des données, y compris le nombre de lignes et de colonnes, les types de données et les statistiques descriptives de base.

* Nettoyer les données: 
Ensuite on a effectué des étapes de nettoyage des données pour traiter les valeurs manquantes, les erreurs et les anomalies dans les données.

* Visualiser les données: 
On a utilisé des bibliothèques de visualisation telles que matplotlib ou seaborn pour créer des graphiques et des visualisations pour explorer les données et examiner les relations entre les variables.

In [ ]:
# Analyse exploratoire des données
def describe_data(data):
    print(data.describe())
describe_data(data)

# Le code suivant nous permet la visualisation de données.
la visualisation est un outil puissant pour l’exploration et l’analyse des données, ainsi que pour la communication des résultats d’une analyse. Elle peut aider à prendre des décisions éclairées en fournissant une compréhension claire et visuelle des données.

In [ ]:
# Visualisation des données
def plot_data(data):
    sns.barplot(data)
    
plt.figure(figsize=(20,5))
plt.title("Les données concernant la qualité du vin par rapport aux composants")
plot_data(data)
plt.show()

La fonction suivante, appelée split_data, prend en entrée trois arguments: data, target_column et test_size. data représente un DataFrame pandas contenant les données à diviser en ensembles d’entraînement et de test. target_column est le nom de la colonne dans data qui contient les valeurs de la variable cible (ou dépendante). test_size est un nombre décimal entre 0 et 1 qui représente la proportion des données à utiliser pour l’ensemble de test. La valeur par défaut de test_size est 0.2, ce qui signifie que 20% des données seront utilisées pour l’ensemble de test et 80% pour l’ensemble d’entraînement.

* La fonction commence par séparer les données en variables indépendantes (ou caractéristiques) et variable dépendante (ou cible). Les variables indépendantes sont stockées dans X, qui est créé en supprimant la colonne target_column du DataFrame d’entrée data. La variable dépendante est stockée dans y, qui est créée en sélectionnant uniquement la colonne target_column du DataFrame d’entrée data.

* Ensuite, la fonction utilise la fonction train_test_split du module sklearn.model_selection pour diviser les données en ensembles d’entraînement et de test. Les arguments d’entrée de cette fonction sont les variables indépendantes (X), la variable dépendante (y) et la taille de l’ensemble de test (test_size). La fonction renvoie quatre tableaux: les variables indépendantes pour l’ensemble d’entraînement (X_train), les variables indépendantes pour l’ensemble de test (X_test), la variable dépendante pour l’ensemble d’entraînement (y_train) et la variable dépendante pour l’ensemble de test (y_test).

* Enfin, la fonction renvoie ces quatre tableaux comme résultat. En résumé, cette fonction divise un DataFrame pandas en ensembles d’entraînement et de test pour les variables indépendantes et dépendantes.

In [73]:
# Modélisation
def split_data(data, target_column, test_size=0.2):
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return X_train, X_test, y_train, y_test

In [74]:
#Appelle de lafonction split_data
X_train, X_test, y_train, y_test = split_data(data, 'quality', test_size=0.3)

* La fonction 'train_model'  fonction entraîne un modèle de régression linéaire en utilisant 
 les données d’entraînement fournies en entrée et renvoie le modèle entraîné.

In [75]:
def train_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

In [76]:
#Appelle de la fonction train
model = train_model(X_train, y_train)

In [77]:
# Entraîner et tester le modèle
def Evaluation_modele(model, X_train, X_test, y_train, y_test):
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    return train_score, test_score

In [78]:
#Appelle de la fonction Evaluation
train_score, test_score = Evaluation_modele(model, X_train, X_test, y_train, y_test)
train_score, test_score

(0.35428488856231544, 0.3504229724199335)

In [ ]:

print(f'Le score d’entraînement du modèle est: {train_score}')
print(f'Le score de test du modèle est: {test_score}')

# le score d’entraînement du modèle est de 0.35 et le score de test est également de 0.35. Cela indique que le modèle a une performance similaire sur les données d’entraînement et de test.

# Afficher les prédictions du modèle en utilisant la méthode predict du modèle sur les données de test X_test

In [93]:
y_pred = model.predict(X_test)

# Les valeurs de MSE et MAE sont respectivement de 0.44 et 0.53, ce qui indique que les prédictions du modèle ont une erreur moyenne d’environ 0.44 et 0.53 par rapport aux étiquettes réelles. La valeur de R² est de 0.35, ce qui indique que les prédictions du modèle s’ajustent modérément bien aux données

In [ ]:
print('MSE:', mean_squared_error(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R²:', r2_score(y_test, y_pred))
